In [24]:
import pandas as pd
from google.cloud import bigquery
import numpy as np

In [25]:
new_output_df = pd.read_csv('input/research_papers_new_processing.csv', index_col=0)
bq_client = bigquery.Client(project='evident-data-dev')
old_output_df = bq_client.list_rows('evident-data-dev.curated_google_scholar.banking_research').to_dataframe()

qa_df = pd.read_csv('input/research_papers_qa_dataset.csv', index_col=0)

c:\Users\lucas\anaconda3\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
c:\Users\lucas\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
C:\Users\lucas\AppData\Local\Temp\ipykernel_14560\2607790838.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  qa_df = pd.read_csv('input/research_papers_qa_dataset.csv', index_col=0)


In [26]:
mask = ~old_output_df['paper_url'].isin(new_output_df['paper_url'])

old_not_new_df = old_output_df[mask]
old_not_new_df.drop_duplicates(subset=['paper_url'], inplace=True)

print(old_not_new_df[old_not_new_df.columns[1]].count())

qa_df

933


C:\Users\lucas\AppData\Local\Temp\ipykernel_14560\1418414822.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_not_new_df.drop_duplicates(subset=['paper_url'], inplace=True)


,last_run_date,author_url,url,title,authors,number_of_authors,publication_date,description,publication_year,citations,year,description_lower,is_patent,title_description,tags,excluded_tags,keyword_tags,has_publication_info,ex_keyword_tags,author_name
0,2025-07-19,/citations?hl=en&user=2FbkAzYAAAAJ,/citations?view_op=view_citation&hl=en&user=2F...,symmetry-informed marl: a decentralized and co...,"Rongye Shi, Xin Yu, Yandong Wang, Yongkai Tian...",8.0,2025/3/21,NaN,2025.0,0,2025.0,NaN,False,NaN,[],[],[],True,[],NaN
1,2025-07-19,/citations?hl=en&user=JRl3iYIAAAAJ,/citations?view_op=view_citation&hl=en&user=JR...,systems and methods for regularizing machine l...,NaN,NaN,2025/3/27,a model Given the dynamic nature of the real w...,2025.0,0,2025.0,a model given the dynamic nature of the real w...,True,systems and methods for regularizing machine l...,['ai_keywords'],[],['machine learning'],True,[],NaN
2,2024-10-01,/citations?hl=en&user=g3MAaQkAAAAJ,/citations?view_op=view_citation&hl=en&user=g3...,"health, habitat and underdevelopment in nigeri...",Tade Akin Aina,1.0,1990,A technical report centering on a study on hou...,1990.0,7,1990.0,a technical report centering on a study on hou...,False,"health, habitat and underdevelopment in nigeri...",[],[],[],True,[],NaN
3,2025-05-06,/citations?hl=en&user=ltO7vTAAAAAJ,/citations?view_op=view_citation&hl=en&user=lt...,telegraph resistance noise of tunnel junctions...,"VI Kozub, AM Rudin",2.0,1994/3/20,Activation processes for a two-level fluctuato...,1994.0,2,1994.0,activation processes for a two-level fluctuato...,False,telegraph resistance noise of tunnel junctions...,[],[],[],True,[],Alexander Rudin
4,2025-05-06,/citations?hl=en&user=-z0a1wUAAAAJ,/citations?view_op=view_citation&hl=en&user=-z...,parcel-1: a parallel programming language base...,"Stéphane Vialle, Thierry Cornu, Yannick Lallement",3.0,1996/8/1,"In this paper we present a new language, calle...",1996.0,9,1996.0,"in this paper we present a new language, calle...",False,parcel-1: a parallel programming language base...,['ai_keywords'],[],['autonomous'],True,[],yannick lallement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298380,2023-10-02,/citations?hl=en&user=O55AvPIAAAAJ,/citations?view_op=view_citation&hl=en&user=O5...,dark energy survey year 3 results: redshift ca...,"Giulia Giannini, A Alarcon, M Gatti, A Porredo...",115.0,2024/1,We present an alternative calibration of the ...,2024.0,0,2022.0,we present an alternative calibration of the ...,False,dark energy survey year 3 results: redshift ca...,['ai_keywords'],[],['clustering'],True,[],NaN
298381,2025-05-06,/citations?hl=en&user=OWrALrAAAAAJ,/citations?view_op=view_citation&hl=en&user=OW...,empirical equilibria in agent-based economic s...,"Kshama Dwarakanath, Svitlana Vyetrenko, Tucker...",3.0,2024/8/21,We present an agent-based simulator for econom...,2024.0,1,2024.0,we present an agent-based simulator for econom...,False,empirical equilibria in agent-based economic s...,['ai_keywords'],[],"['artificial intelligence', 'reinforcement lea...",True,[],Svitlana Vyetrenko
298382,2025-05-06,/citations?hl=en&user=nMtXBtQAAAAJ,/citations?view_op=view_citation&hl=en&user=nM...,integrating ai's carbon footprint into risk ma...,Nataliya Tkachenko,1.0,2024/9/15,This paper examines the integration of AI's ca...,2024.0,0,2024.0,this paper examines the integration of ai's ca...,False,integrating ai's carbon footprint into risk ma...,['ai_keywords'],[],[' ai '],True,[],"Tkachenko Nataliya, PhD, MA"
298383,2025-05-06,/citations?hl=en&user=5e-gHjMAAAAJ,/citations?view_op=view_citation&hl=en&user=5e...,dishonest majority multi-verifier zero-knowled...,"Daniel Escudero, Antigoni Polychroniadou, Yifa...",4.0,2024,In this work we study the efficiency of Zero-K...,2024.0,2,2024.0,in this work we study the efficiency of zero-k...,False,dishonest majority multi-verifier zero-knowled...,[],[],[],True,[],Antigoni Polychroniadou


In [27]:
lhs_cols = ["company_name", "paper_url"]
rhs_cols = ["authors", 'url', "title", "publication_date", "description", "is_patent", "tags",
            "excluded_tags", "keyword_tags", "has_publication_info", "author_name"]

In [28]:
banks_with_qa = pd.merge(
    old_not_new_df[lhs_cols],
    qa_df[rhs_cols],
    left_on='paper_url',
    right_on = 'url',
    how='left').drop(columns=['url'])

banks_with_qa = banks_with_qa.replace("[]", np.nan)

def get_exclusion_reason(row):
    reasons = []

    if row['is_patent'] == True:
        reasons.append("is_patent")

    if pd.notna(row['excluded_tags']):
        reasons.append("excluded_tags")

    if pd.isna(row['tags']):
        reasons.append("no_tags")

    if pd.isna(row['authors']):
        reasons.append("no_authors")

    if pd.isna(row['publication_date']):
        reasons.append("no_publication_date")

    if pd.isna(row['author_name']):
        reasons.append("no_author_name")

    return reasons


banks_with_qa['exclusion_reason'] = banks_with_qa.apply(get_exclusion_reason, axis=1)
banks_with_qa.drop(columns=['tags', 'has_publication_info', 'publication_date', 'is_patent', 'keyword_tags'], inplace=True)
banks_with_qa.drop_duplicates(subset=['paper_url'], inplace=True)

banks_with_qa

,company_name,paper_url,authors,title,description,excluded_tags,author_name,exclusion_reason
0,Commerzbank,/citations?view_op=view_citation&hl=en&user=TD...,"Ludvik Tesar, Hidefumi Kobatake, Akinobu Shimi...",gaussian mixture model for texture-based medic...,"In this paper, a method for medical diagnosing...",NaN,Ludvík Tesař,[no_publication_date]
1,Santander,/citations?view_op=view_citation&hl=en&user=mZ...,"José M Leiva-Murillo, Antonio Artés-Rodrıguez",algorithms for maximum-likelihood bandwidth,"Abstract 8 In machine learning and statistics,...",NaN,Jose M. Leiva Murillo,[no_publication_date]
2,JPMorganChase,/citations?view_op=view_citation&hl=en&user=-N...,"Sarbaree Mishra, V Komandla, S Bandi, S Konida...",training ai models on sensitive data-the feder...,NaN,NaN,Sairamesh Konidala,[no_tags]
3,JPMorganChase,/citations?view_op=view_citation&hl=en&user=-N...,Sairamesh Konidala,designing and implementing efficient data pipe...,NaN,NaN,Sairamesh Konidala,[no_tags]
4,JPMorganChase,/citations?view_op=view_citation&hl=en&user=1W...,"Ryan S DeFever, Colin Targonski, Steven W Hall...",a generalized deep learning approach for local...,Identifying local structure in molecular simul...,['excluded_keywords'],Colin Targonski,[excluded_tags]
...,...,...,...,...,...,...,...,...
1238,CommBank,/citations?view_op=view_citation&hl=en&user=V8...,"Naime Ranjbar Kermany, Weiliang Zhao, Jian Yan...",incorporating user rating credibility in recom...,There has been lots of research effort aimed a...,NaN,Naomi (Naime) Ranjbar Kermany,[no_publication_date]
1239,CommBank,/citations?view_op=view_citation&hl=en&user=Xk...,"Harris Borman, Anna Leontjeva, Luiz Pizzato, M...",do llm personas dream of bull markets? compari...,Large Language Models (LLMs) have demonstrated...,NaN,Anna Leontjeva,[no_publication_date]
1241,ANZ,/citations?view_op=view_citation&hl=en&user=Si...,SHAMPA SHAHRIYAR,inherently edge preserving depth map coding us...,There has been a growing research interest in ...,NaN,Shampa Shahriyar,[no_publication_date]
1245,Westpac,/citations?view_op=view_citation&hl=en&user=fl...,Luci Ellis,"lumps, bumps and waves",Thank you to the Ai Group for the invitation t...,NaN,Luci Ellis,[no_publication_date]


In [29]:
banks_with_qa.to_csv("output/papers_not_in_new.csv")